# Keras on Session mode

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical

import numpy as np
import matplotlib.pyplot as plt
import os

#import tensorflow.contrib.eager as tfe
#tf.enable_eager_execution()

In [2]:
learning_rate = 0.001
training_epochs = 21
batch_size = 100

In [3]:
cur_dir = os.getcwd()
ckpt_dir_name = 'checkpoints'
model_dir_name = 'mnist_basic_seq'

checkpoint_dir = os.path.join(cur_dir, ckpt_dir_name, model_dir_name)
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir_name)

## TensorBoard
log_dir = os.path.join(cur_dir, 'tensorboard', model_dir_name)
os.makedirs(log_dir, exist_ok=True)

In [4]:
mnist = keras.datasets.mnist

(train_x, train_y), (test_x, test_y) = mnist.load_data()
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [5]:
train_x = train_x.astype(np.float32) / 255
test_x = test_x.astype(np.float32) / 255

train_x = np.reshape(train_x, [-1,28*28])
test_x = np.reshape(test_x, [-1,28*28])

train_y = to_categorical(train_y, 10) ## one-hot encoding
test_y = to_categorical(test_y,10)

# tf.data.Dataset

train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).shuffle(buffer_size=70000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).batch(batch_size)

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(60000, 784) (60000, 10) (10000, 784) (10000, 10)


## Sequential

## Functional model

## Class model